In [3]:
import json

import requests
from flask import Flask, request, render_template
from wtforms import Form, TextField, validators, TextAreaField
import jsonpickle

import config
from document import Document

indexer_url = config.indexer_url
text_processing_url = config.text_processing_url
ranking_url = config.ranking_url
snippets_url = config.snippets_url
res_page_form_url = config.res_page_form_url


In [4]:
class ReusableForm(Form):
    query = TextField('Search Query: ', validators=[validators.required()])
    skills = TextAreaField('Enster skils: ')


app = Flask(__name__)


@app.route('/', methods=['GET', 'POST'])
def index():
    """
    If GET -> renders Main page from templates/index.html
    If POST -> renders ranged search results
    """
    form = ReusableForm(request.form)
    if request.method == 'POST':
        query = request.form['query']
        
        # Normilize search query
        r = requests.post(text_processing_url + config.NORMALIZE_QUERY_PATH,
                          json=query)
        search_query = r.text
        
        # Search
        r = requests.post(indexer_url + config.SEARCH_PATH,
                          json=search_query)
        search_result = jsonpickle.decode(r.text)
        documents = search_result["documents"]
        terms = search_result["terms"]
        
        # Ranking
        search_result = {"documents": documents, "query": search_query}
        r = requests.post(ranking_url + config.RANK_PATH,
                          json=jsonpickle.encode(search_result))
        search_result = jsonpickle.decode(r.text)
        
        # Get snippets
        search_result = {"documents": [i[1] for i in search_result],
                         "terms": terms}
        r = requests.post(snippets_url + config.SNIPPETS_PATH,
                          json=jsonpickle.encode(search_result))
        search_result = jsonpickle.decode(r.text)
        
        # SERP
        search_result = {"documents": search_result, "query": search_query}
        r = requests.post(res_page_form_url + config.RESULT_PAGE_PATH,
                          json=jsonpickle.encode(search_result))
        result_page = r.text
        return result_page
    else:
        return render_template('main_page.html', form=form)


/home/vzalevskyi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Required is going away in WTForms 3.0, use DataRequired
  


In [ ]:
if __name__ == "__main__":
    app.run(host=config.MANAGER_HOST, port=config.MANAGER_PORT)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:13505/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Dec/2018 01:02:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2018 01:02:07] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2018 01:02:08] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2018 01:02:19] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2018 01:02:19] "GET /favicon.ico HTTP/1.1" 404 -
